In [1]:
import os
import pandas as pd
import zipfile

In [2]:
def read_csvs_from_zip(zip_path: str, csv_path: str, *args, **kwargs) -> pd.DataFrame:
    with zipfile.ZipFile(zip_path) as zip:
        with zip.open(csv_path) as csv:
            data = pd.read_csv(csv, *args, **kwargs)
    return data

Load `children_in_poverty` data

In [3]:
children_in_poverty = pd.read_excel(
    '../../data/raw/neet-factors/children-in-low-income-families-local-area-statistics-2014-to-2022.ods',
    sheet_name='4_Absolute_Local_Authority',
    skiprows=9,
    usecols=[1, 17],
    names=['geography_code', 'Children in poverty'],
    index_col=0
).iloc[:, 0].mul(100).round(1)

Load `children_looked_after` data

In [4]:
children_looked_after = read_csvs_from_zip(
          '../../data/raw/neet-factors/cla-all.zip',
          'data/cla_number_and_rate_per_10k_children.csv',
          usecols=[
            'time_period',
            'geographic_level',
            'population_count',
            'new_la_code',
            'rate_per_10000'
          ],
          index_col='new_la_code'
        )

children_looked_after = children_looked_after.loc[
  (children_looked_after.geographic_level == 'Local authority') &
  (children_looked_after.population_count == 'Children looked after at 31 March each year') &
  (children_looked_after.time_period == 2022),
  ["rate_per_10000"]
].rename(columns={
  'rate_per_10000': 'Children looked after'
})

Load `health_disability` data

In [5]:
health_disability_2021 = pd.read_csv('../../data/raw/neet-factors/health_disability_2021.csv', index_col='Lower tier local authorities Code')

sum = health_disability_2021.loc[
  (health_disability_2021['Disability (3 categories)'] == 'Disabled under the Equality Act') &
  (health_disability_2021['Age (C) (4 categories)'].isin(['Aged 15 years and under', 'Aged 16 to 24 years' ])),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

total = health_disability_2021.loc[
  (health_disability_2021['Age (C) (4 categories)'].isin(['Aged 15 years and under', 'Aged 16 to 24 years' ])),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

health_disability_2021 = (sum/total).mul(100).to_frame('Disability (age < 25)')

Load `family_disability_2021` data

In [6]:
family_disability_2021 = pd.read_csv(
  filepath_or_buffer='../../data/raw/neet-factors/family_disability_2021.csv',
  index_col='Lower tier local authorities Code'
)

sum = family_disability_2021.loc[
  family_disability_2021['Disability - Equality act disabled (4 categories) Code'].isin([1]),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

total = family_disability_2021.loc[
  :,
  "Observation"
].groupby('Lower tier local authorities Code').sum()

family_disability_2021 = (sum/total).mul(100).to_frame('Disability (all)')

Load `economic_inactivity` data.

This is not the same value as in the spreadsheet. Need to double check logic.

In [7]:
economic_inactivity = pd.read_csv(
  '../../data/raw/neet-factors/economic_inactivity_status.csv',
  index_col='Lower tier local authorities Code'
)

in_age_range = economic_inactivity['Age (C) (4 categories)'] == 'Aged 16 to 24 years'
is_economically_inactive = economic_inactivity['Economic activity status (7 categories)'] == 'Economically inactive (excluding full-time students)'

val = economic_inactivity.loc[in_age_range & is_economically_inactive, 'Observation'].groupby('Lower tier local authorities Code').sum()
total = economic_inactivity.loc[in_age_range, 'Observation'].groupby('Lower tier local authorities Code').sum()

economic_inactivity = ( val / total ).mul(100).to_frame('Economic inactivity (NEET)')

Load `fertility` data

In [8]:
fertility = pd.read_csv(
    '../../data/raw/neet-factors/fertility_rates.csv', index_col=[
        'GEOGRAPHY_CODE',
        'MEASURE_NAME',
    ], usecols=[
        'GEOGRAPHY_CODE',
        'MEASURE_NAME',
        'OBS_VALUE',
        # 'OBS_STATUS',
        # 'OBS_STATUS_NAME',
        # 'OBS_CONF',
        # 'OBS_CONF_NAME',
    ]
)

fertility = fertility.squeeze().unstack().rename(columns={
    'Age specific fertility rate : Aged under 20': 'Fertility rates (age < 20)',
    'Age specific fertility rate : Aged 20-24': 'Fertility rates (age 20-24)'
})

Create `imd` data - crime and health

In [9]:
imd_health = pd.read_excel(
  '../../data/raw/neet-factors/File_10_-_IoD2019_Local_Authority_District_Summaries__lower-tier__.xlsx',
  sheet_name='Health',
  usecols='A,E',
  names=['geography_code', 'IMD Health'],
  index_col='geography_code',
)
imd_crime = pd.read_excel(
  '../../data/raw/neet-factors/File_10_-_IoD2019_Local_Authority_District_Summaries__lower-tier__.xlsx',
  sheet_name='Crime',
  usecols='A,E',
  names=['geography_code', 'IMD Crime'],
  index_col='geography_code',
)

Load `lone_parent_household` data

In [10]:
lone_parent_households = pd.read_excel(
    '../../data/raw/neet-factors/aps2004to2019finalv2.xlsx',
    sheet_name='Lone_parent_households',
    usecols="A,T",
    skiprows=10,
    names=['geography_code', 'lone_parent_households'],
    index_col='geography_code',
    na_values=['[u]', '[w]'],
).join(pd.read_excel(
    '../../data/raw/neet-factors/aps2004to2019finalv2.xlsx',
    sheet_name='Total_households',
    usecols="A,T",
    skiprows=10,
    names=['geography_code', 'total_households'],
    index_col='geography_code',
    na_values=['[u]', '[w]'],
))

lone_parent_households = (lone_parent_households.lone_parent_households / lone_parent_households.total_households).mul(100).to_frame('Lone parent households')

Add `pupils_with_sen_support` data

In [11]:
def sen_filter(data):
    return data.loc[
        (data.time_period == 202223)
        & (data.geographic_level == 'Local authority')
        & (data.hospital_school == 'Total')
        & (data.type_of_establishment == 'Total')
        & (data.phase_type_grouping == 'Total')
        ,
        [
          'sen_support_percent'
        ]
    ]
pupils_with_sen_support = read_csvs_from_zip(
  '../../data/raw/neet-factors/special-educational-needs-in-england_2022-23.zip',
  csv_path='data/sen_phase_type_.csv',
  index_col=[
      'new_la_code',
  ],
  na_values=['z']
).pipe(sen_filter).rename(columns={'sen_support_percent': 'Pupils with SEN support'}).sort_index()

Load `qualifications` data

In [12]:
def filter_qualifications_16_24(d: pd.DataFrame) -> pd.DataFrame:
    return d.loc[
      (d['Age (C) (4 categories)'] == 'Aged 16 to 24 years'),
      [
        'Highest level of qualification (7 categories) Code',
        'Observation'
      ]
    ]

def filter_qualifications_25_plus(d: pd.DataFrame) -> pd.DataFrame:
    return d.loc[
      (d['Age (C) (4 categories)'].isin(['Aged 25 to 49 years', 'Aged 50 years and over'])),
      [
        'Highest level of qualification (7 categories) Code',
        'Observation'
      ]
    ]

def calc_pct_below_level_2(d):
    return (
      d.loc[
        d['Highest level of qualification (7 categories) Code'].between(0, 1, inclusive='both')
      ].groupby('Lower tier local authorities Code').Observation.sum()
      /
      d.groupby('Lower tier local authorities Code').Observation.sum()
    ).mul(100)

qualifications_16_24 = pd.read_csv(
  '../../data/raw/neet-factors/qualifications_below_level_2.csv',
  index_col=[
    'Lower tier local authorities Code',
  ]
).pipe(filter_qualifications_16_24).pipe(calc_pct_below_level_2).to_frame('Qualification below level 2 (age 16-24)')

qualifications_all = pd.read_csv(
  '../../data/raw/neet-factors/qualifications_below_level_2.csv',
  index_col=[
    'Lower tier local authorities Code',
  ]
).pipe(filter_qualifications_25_plus).pipe(calc_pct_below_level_2).to_frame('Qualification below level 2 (all)')

Add school absences data

In [13]:
school_absences = read_csvs_from_zip(
  '../../data/raw/neet-factors/school_absence_by_geographic_area.zip',
  csv_path='data/1_absence_3term_nat_reg_la.csv',
  index_col='new_la_code',
).query('time_period == 202122 & geographic_level == "Local authority" & school_type == "Total"')

school_absences = (school_absences.sess_overall / school_absences.sess_possible).mul(100).sort_index().to_frame('School Absences')

Add school suspension or exclusion data

In [14]:
school_data = (
    read_csvs_from_zip(
        '../../data/raw/neet-factors/permanent-and-fixed-period-exclusions-in-england_2022-23-autumn-term.zip',
        csv_path='data/exc_school_20230808.csv',
        index_col='new_la_code',
        usecols=[
          'new_la_code',
          'time_period',
          'headcount',
          'perm_excl',
          'suspension',
        ]
    )
    .query(
        'time_period == 202122'
    )
    .loc[
        :, ['headcount', 'perm_excl', 'suspension']
    ]
    .groupby('new_la_code')
    .sum()
    .sort_index()
)

school_data = pd.concat(
  [
    (school_data.perm_excl / school_data.headcount).mul(100),
    (school_data.suspension / school_data.headcount).mul(100)
  ],
  axis=1
).rename(columns={
    0: 'School Exclusions',
    1: 'School Suspensions'
  },
)

Add social renting data

In [15]:
def calc_pct_social_rented(d):
    numerator = d.loc[d['Tenure of household (9 categories)'].str.startswith('Social rented')].groupby('Lower tier local authorities Code').Observation.sum()
    denominator = d.groupby('Lower tier local authorities Code').Observation.sum()
    return (numerator / denominator).mul(100).to_frame('Socially renting households')

social_renting = pd.read_csv(
  '../../data/raw/neet-factors/socially_renting_households.csv',
  index_col='Lower tier local authorities Code',
).pipe(calc_pct_social_rented)

Add caring responsibility data

In [16]:
def filter_young_people(d):
    return d.loc[d['Age (6 categories)'] == 'Aged 16 to 24 years']

def calc_pct(d: pd.DataFrame):
    numerator = d.loc[d['Unpaid care (5 categories) Code'].ge(2)].groupby('Lower tier local authorities Code').Observation.sum()
    denominator = d.groupby('Lower tier local authorities Code').Observation.sum()
    return (numerator / denominator).mul(100).to_frame('Unpaid carer (age 16-24)')

unpaid_carer = pd.read_csv(
  '../../data/raw/neet-factors/unpaid_carer.csv',
  index_col='Lower tier local authorities Code',
).pipe(filter_young_people).pipe(calc_pct)

Create base data frame

In [17]:
local_authorities = pd.read_csv('../../data/reference/local_authorities.csv', index_col=[0])

Collate all layers into a single file

In [18]:
data = (
  local_authorities
    .join(children_in_poverty)
    .join(children_looked_after)
    .join(health_disability_2021)
    .join(family_disability_2021)
    .join(economic_inactivity)
    .join(fertility)
    .join(imd_health)
    .join(imd_crime)
    .join(lone_parent_households)
    .join(pupils_with_sen_support)
    .join(qualifications_16_24)
    .join(qualifications_all)
    .join(school_absences)
    .join(school_data)
    .join(social_renting)
    .join(unpaid_carer)
    .set_index(['Local Authority Name', 'Group'], append=True)
  )

Save to a CSV file

In [19]:
SOURCES_CSV='../../data/processed/yff/neet-factors-sources.csv'
os.makedirs(os.path.dirname(SOURCES_CSV), exist_ok=True)
data.melt(ignore_index=False).to_csv(SOURCES_CSV)

Data that seems wrong:

* Economic inactivity (NEET) - big difference
* Fertility rates (20-24) - small difference
* Qualification below level 2 (all)

In [23]:
data

,,,Children in poverty,Children looked after,Disability (age < 25),Disability (all),Economic inactivity (NEET),Fertility rates (age 20-24),Fertility rates (age < 20),IMD Health,IMD Crime,Lone parent households,Pupils with SEN support,Qualification below level 2 (age 16-24),Qualification below level 2 (all),School Absences,School Exclusions,School Suspensions,Socially renting households,Unpaid carer (age 16-24)
Local Authority Code,Local Authority Name,Group,,,,,,,,,,,,,,,,,,
E06000001,Hartlepool,"Services, Manufacturing and Mining Legacy",21.7,155,10.124733,11.311311,12.079055,66.5,19.2,0.869,0.566,11.085973,15.281077,24.069780,33.359578,7.947627,0.104808,14.181842,23.440592,6.095155
E06000002,Middlesbrough,Manufacturing Traits,31.8,150,8.978449,10.800000,11.562345,68.4,19.3,1.194,0.701,15.806988,14.169557,23.646299,35.188001,8.776107,0.177431,27.411087,23.240687,5.045736
E06000003,Redcar and Cleveland,"Services, Manufacturing and Mining Legacy",22.0,125,10.040150,10.100000,11.305419,71.1,20.9,0.790,0.138,9.841270,14.431018,25.320197,32.094161,7.883450,0.306777,24.216789,18.899056,5.750144
E06000004,Stockton-on-Tees,"Services, Manufacturing and Mining Legacy",19.0,133,9.688022,9.109109,10.400090,63.2,14.9,0.684,-0.111,10.440835,12.675524,23.853056,28.419422,7.732062,0.118023,10.858154,16.324801,5.216711
E06000005,Darlington,"Services, Manufacturing and Mining Legacy",20.5,123,9.393366,8.391608,9.726138,65.5,17.3,0.532,0.558,11.205074,13.776228,24.413571,28.626379,7.674965,0.187829,11.106937,16.213840,5.257349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E09000029,Sutton,Suburban Traits,8.3,48,7.783485,6.400000,7.041275,42.3,6.0,-0.570,-0.174,16.919192,9.569244,19.199102,24.776830,6.916140,0.110687,4.860411,14.168094,4.413100
E09000030,Tower Hamlets,London Cosmopolitan,21.8,52,6.975956,10.400000,6.456436,23.3,3.2,0.195,0.509,7.588933,12.400140,13.133683,25.217391,6.150389,0.019569,2.800487,35.869719,3.889538
E09000031,Waltham Forest,Ethnically Diverse Metropolitan Living,16.8,53,5.819617,7.700000,7.838366,41.5,6.6,-0.161,0.406,14.522059,11.529270,20.202765,27.408201,6.770535,0.059170,4.371494,21.475982,4.554707
